In [ ]:
import stacrs
from pystac import Item
from odc.stac import load

from odc.geo import Geometry
from odc.geo.xr import mask
from dask.distributed import Client

import geopandas as gpd

In [ ]:
url = (
    "https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/"
    "9469f09592ced973a3448cf66b6100b741b64c0d/releaseData/gbOpen/BLZ/ADM0/geoBoundaries-BLZ-ADM0-all.zip"
)

geometry = gpd.read_file(url, layer="geoBoundaries-BLZ-ADM0")
geometry.explore()

In [ ]:
url = "https://data.ldn.auspatious.com/geo_ls_lp/geo_ls_lp_0_1_0.parquet"

dict_list = stacrs.search(
    url,
    bbox=list(geometry.geometry.bounds.values.flatten())
)

items = [Item.from_dict(d) for d in dict_list]

data = load(items, geopolygon=geometry, chunks={})
data

In [ ]:
masked = mask(data, Geometry(geometry.to_geo_dict()))

In [ ]:
masked.isel(time=-1).evi2.odc.explore()

In [ ]:
with Client(n_workers=4, threads_per_worker=16) as client:
    annual_mean = masked.evi2.mean(dim=["longitude", "latitude"]).compute()

In [ ]:
import matplotlib.pyplot as plt

annual_mean.plot(size=3, aspect=2.5)
_ = plt.title("Annual Mean EVI2")